# FUNCIONES

In [175]:
import os
import numpy as np
import copy
from math import sqrt
from scipy.linalg import solve_triangular

### Creación matrices

In [176]:
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias

    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    #Se inicializa una matriz llena de ceros con las dimensiones deseadas (mxn)
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            #Si entero es verdadero se obtiene el maximo entero menor o igual a 1 (//1)
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

crea_matriz.__doc__ ="Descripción:\n"\
"        Función que genera matrices de tamaño definido con entradas dentro del rango indicado\n"\
"\n"\
"Parametros:\n"\
"        renglones       integer indica renglones de la matriz\n"\
"        columnas        integer indica columnas de la matriz\n"\
"        maximo_valor    integer o double señala valor máximo de las entradas de la matriz\n"\
"        minimo_valor    integer o double señala valor mínimo de las entradas de la matriz\n"\
"        entero          Boolean, indica si las entradas serán enteras (True) o no (False)\n"\
"\n"\
"Resultado:\n"\
"        M               Matriz (mxn) con numeros al azar en el rango indicado\n"\
"\n"\
"Ejemplo de matriz de m=4 renglones y n=3 columnas, con entradas reales entre 3.4 y 15.6\n"\
"crea_matriz(m,n,3.4,15.6)\n"\
"Ejemplo de matriz de m=2 renglones y n=2 columnas, con entradas enteras entre 0 y 1\n"\
"crea_matriz(2,2,0,1,True)\n"

In [177]:
help(crea_matriz)

Help on function crea_matriz in module __main__:

crea_matriz(renglones, columnas, maximo_valor, minimo_valor, entero=False)
    Descripción:
            Función que genera matrices de tamaño definido con entradas dentro del rango indicado
    
    Parametros:
            renglones       integer indica renglones de la matriz
            columnas        integer indica columnas de la matriz
            maximo_valor    integer o double señala valor máximo de las entradas de la matriz
            minimo_valor    integer o double señala valor mínimo de las entradas de la matriz
            entero          Boolean, indica si las entradas serán enteras (True) o no (False)
    
    Resultado:
            M               Matriz (mxn) con numeros al azar en el rango indicado
    
    Ejemplo de matriz de m=4 renglones y n=3 columnas, con entradas reales entre 3.4 y 15.6
    crea_matriz(m,n,3.4,15.6)
    Ejemplo de matriz de m=2 renglones y n=2 columnas, con entradas enteras entre 0 y 1
    crea_

### Factorización QR

In [178]:
def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    componente para obtener matriz de reflexión householder v
            v       vector que representa la reflexión de householder
    """
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

house.__doc__ ="Descripción:\n"\
"        Función que calcula vector de householder\n"\
"\n"\
"Parametros:\n"\
"        x       np.ndarray que representa vector al que se le hará la reflexión householder\n"\
"\n"\
"Resultado:\n"\
"        Beta    double para obtener matriz de reflexión householder v\n"\
"        v       np.ndarray que representa vector que representa la reflexión de householder\n"\
"\n"\
"Con el vector v y el factor beta obtenidos se puede calcular la matriz de reflexión de householder R,\n"\
"donde R= Identidad(mxm) - Beta*v*v_transpuesta (v*v_transpuesta es matriz de pxp si v tiene p entradas)\n"\
"El vector v está construido de tal forma que al generar la matriz de reflexión de householder Rf y\n"\
"multiplicarse por x se obtiene una proyección de este último sobre el vector canónico e_1=(1,0,0,...,0)\n"\
"obteniendo un vector con todas sus entradas ingualas a cero excepto la primera, que de hecho es.\n"\
"que de hecho es ||x||e_1\n"\
"\n"\
"Esta mecánica se puede implementar en matrices para generar matrices triangulares superiores (generando ceros en\n"\
"el triangulo inferior) como se ilustra a continuación\n"\
"\n"\
"   [ a00 a01 a02 a03 ]\n"\
"A= [ a10 a11 a12 a13 ]\n"\
"   [ a20 a21 a22 a23 ]\n"\
"   [ a30 a31 a32 a33 ]\n"\
"Tomese x=A[:,0]=[a00,a10,a20,a30], calcular (v,Beta) y con ello la matriz de reflexión Rf_0. Con el producto Rf_0*A\n"\
"se obtiene una matriz de la forma:\n"\
"   [  *   *   *   *  ]\n"\
"A= [  0   *   *   *  ]\n"\
"   [  0   *   *   *  ]\n"\
"   [  0   *   *   *  ]\n"\
"Tomese x=A[:,1]=[a11,a21,a31], calcular (v,Beta) y con ello la matriz de reflexión Rf_1. Con el producto Rf_1*Rf_0*A\n"\
"se obtiene una matriz de la forma:\n"\
"   [  *   *   *   *  ]\n"\
"A= [  0   *   *   *  ]\n"\
"   [  0   0   *   *  ]\n"\
"   [  0   0   *   *  ]\n"\


In [179]:
help(house)

Help on function house in module __main__:

house(x)
    Descripción:
            Función que calcula vector de householder
    
    Parametros:
            x       np.ndarray que representa vector al que se le hará la reflexión householder
    
    Resultado:
            Beta    double para obtener matriz de reflexión householder v
            v       np.ndarray que representa vector que representa la reflexión de householder
    
    Con el vector v y el factor beta obtenidos se puede calcular la matriz de reflexión de householder R,
    donde R= Identidad(mxm) - Beta*v*v_transpuesta (v*v_transpuesta es matriz de pxp si v tiene p entradas)
    El vector v está construido de tal forma que al generar la matriz de reflexión de householder Rf y
    multiplicarse por x se obtiene una proyección de este último sobre el vector canónico e_1=(1,0,0,...,0)
    obteniendo un vector con todas sus entradas ingualas a cero excepto la primera, que de hecho es.
    que de hecho es ||x||e_1
    
    

In [180]:
x=np.array([1,2,3.])
print("x=",x)

Beta,v=house(x)
print("\nBeta=",np.round(Beta,4))
print("\nv=",np.round(v,4))

#Calculando matriz de reflexión de householder R
R=np.eye(3)-Bet*np.outer(ve,ve)
print("\nR=",np.round(R,4))

#Observese como el producto de Rx da por resultado un vector con
#todas sus entradas igualas a cero excepto la primera
print("\nRx=",np.round(R@x,4))

x= [1. 2. 3.]

Beta= 0.7327

v= [ 1.     -0.7295 -1.0942]

R= [[ 0.2673  0.5345  0.8018]
 [ 0.5345  0.6101 -0.5849]
 [ 0.8018 -0.5849  0.1227]]

Rx= [3.7417 0.     0.    ]


In [181]:
def matriz_Arv(A):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la matriz R y las entradas  
    de los vectores householder v (excepto la primera), con los cuales se puede calcular la matriz Q. Ambas matrices
    componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R y los vectores
                   householder con los que se puede obtener la matriz Q, y con ello la factorización QR
    """
    #m contiene el numero de renglones y n el de columnas
    m=A.shape[0]
    n=A.shape[1]
    #se crea una matriz con los valores de A
    Arv=copy.copy(A)
    for j in range(n):
        beta, v=house(Arv[j:m,j])
        #Con esta multiplicación se van generando las componentes r de la matriz R
        Arv[j:m,j:n]=Arv[j:m,j:n]-beta*(np.outer(v,v)@Arv[j:m,j:n])
        #se guarda en cada columnas los valores de v d, excepto la primer componente (que vale 1)
        Arv[(j+1):m,j]=v[1:(m-j)]
    return Arv

matriz_Arv.__doc__ ="Descripción:\n"\
"        Función que genera una matriz que contiene los elementos r distintos de cero de la matriz R y las entradas\n"\
"        de los vectores householder v (excepto la primera), con los cuales se puede calcular la matriz Q. Ambas matrices\n"\
"        componentes de la factorización QR\n"\
"\n"\
"Parametros:\n"\
"        A      numpy.ndarray que representa matriz (mxn) de la que se desea obtener elemento de \n"\
"               factorización QR\n"\
"\n"\
"Resultado:\n"\
"        Arv    numpy.ndarray que representa matriz (mxn) que incluye las componentes distintas de \n"\
"               cero de la matriz R y los vectores householder con los que se puede obtener la matriz \n"\
"               Q, y con ello la factorización QR\n"\
"\n"\
"La factorización QR consiste en encontrar 2 matrices tales que A=Q*R \n"\
"donde Q es una matriz ortogonal y R es una matriz triangular superior \n"\
"Las entradas de la matriz que arroja esta función, Arv, se pueden visualizar como:\n"\
"     [ r11  r12  r13  r14 ]\n"\
"     [ v21  r22  r23  r24 ]\n"\
"Arv= [ v31  v32  r33  r34 ]\n"\
"     [ v41  v42  v43  r44 ]\n"\
"     [ v51  v52  v53  v54 ]\n"\
"\n"\
"Las entradas r conforman la matriz R (triangular superior): \n"\
"     [ r11  r12  r13  r14 ] \n"\
"     [  0   r22  r23  r24 ] \n"\
"R=   [  0    0   r33  r34 ] \n"\
"     [  0    0    0   r44 ] \n"\
"     [  0    0    0    0  ] \n"\
"La matriz Q puede obtenerse a partir de los vectores householder v, los cuales \n"\
"sirven para construir la matriz de reflexión de householder R, con \n"\
"R=Identidad(mxm) - Beta*v*v_trans  y  Beta=2/(v_trans*v) \n"\
"                       [  1                 ]\n"\
"                       [ v21   1            ]\n"\
"vectores householder*= [ v31  v32   1       ]\n"\
"                       [ v41  v42  v43   1  ]\n"\
"                       [ v51  v52  v53  v54 ]\n"\
"*Notese que Avr no tiene la primera entrada de los vectores v, pero sabemos por construcción \n"\
"que es igual a 1"

In [182]:
help(matriz_Arv)

Help on function matriz_Arv in module __main__:

matriz_Arv(A)
    Descripción:
            Función que genera una matriz que contiene los elementos r distintos de cero de la matriz R y las entradas
            de los vectores householder v (excepto la primera), con los cuales se puede calcular la matriz Q. Ambas matrices
            componentes de la factorización QR
    
    Parametros:
            A      numpy.ndarray que representa matriz (mxn) de la que se desea obtener elemento de 
                   factorización QR
    
    Resultado:
            Arv    numpy.ndarray que representa matriz (mxn) que incluye las componentes distintas de 
                   cero de la matriz R y los vectores householder con los que se puede obtener la matriz 
                   Q, y con ello la factorización QR
    
    La factorización QR consiste en encontrar 2 matrices tales que A=Q*R 
    donde Q es una matriz ortogonal y R es una matriz triangular superior 
    Las entradas de la matriz que a

In [183]:
def matriz_R(A_r_v):
    """
    Función que devuelve la matriz R de la factorización QR de una matriz A, 
    apartir de la matriz Arv
    
    params: A_r_v   Matriz (mxn) que incluye elementos de la matriz R y Q, de la factorización QR

    return: R       Matriz (mxn) R de la factorización A=QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    R=np.zeros((m,n))
    #la matriz A_r_v ya tiene los elementos de r en el triangulo superior
    #por lo que únicamente se sustraen dichos valores y lo demás se deja en ceros
    for j in range(n):
        R[0:(j+1),j]=A_r_v[0:(j+1),j]
    return R

matriz_R.__doc__ ="Descripción:\n"\
"        Función que devuelve la matriz R de la factorización QR de una matriz A,\n"\
"        apartir de la matriz Arv\n"\
"\n"\
"Parametros:\n"\
"        A_r_v   np.ndarray que representa matriz (mxn) que incluye elementos de la matriz R y Q,\n"\
"                de la factorización QR\n"\
"\n"\
"Resultado:\n"\
"        R       np.ndarray que representa matriz (mxn) R de la factorización A=QR\n"\
"\n"\
"Visualmente la sustracción de los elementos de R se ve así:\n"\
"     [ r11  r12  r13  r14 ]            [ r11  r12  r13  r14 ]\n"\
"     [ v21  r22  r23  r24 ]            [  0   r22  r23  r24 ]\n"\
"Arv= [ v31  v32  r33  r34 ]   -->   R= [  0    0   r33  r34 ]\n"\
"     [ v41  v42  v43  r44 ]            [  0    0    0   r44 ]\n"\
"     [ v51  v52  v53  v54 ]            [  0    0    0    0  ]\n"

In [184]:
help(matriz_R)

Help on function matriz_R in module __main__:

matriz_R(A_r_v)
    Descripción:
            Función que devuelve la matriz R de la factorización QR de una matriz A,
            apartir de la matriz Arv
    
    Parametros:
            A_r_v   np.ndarray que representa matriz (mxn) que incluye elementos de la matriz R y Q,
                    de la factorización QR
    
    Resultado:
            R       np.ndarray que representa matriz (mxn) R de la factorización A=QR
    
    Visualmente la sustracción de los elementos de R se ve así:
         [ r11  r12  r13  r14 ]            [ r11  r12  r13  r14 ]
         [ v21  r22  r23  r24 ]            [  0   r22  r23  r24 ]
    Arv= [ v31  v32  r33  r34 ]   -->   R= [  0    0   r33  r34 ]
         [ v41  v42  v43  r44 ]            [  0    0    0   r44 ]
         [ v51  v52  v53  v54 ]            [  0    0    0    0  ]



In [185]:
def matriz_Q(A_r_v):
    """
    Función que devuelve la matriz R de la factorización QR de una matriz A,
    apartir de la matriz Arv
    
    params: A_r_v   Matriz (mxn) con la info escencial para la factorización

    return: Q       Matriz Q (mxm) de la factorización A=QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Q=np.eye(m)
    I=np.eye(m)
    for j in range(n-1,-1,-1):
        #Se sustrae la información de los vectores de householder contenida
        #en la matriz Arv, agregando la primera entrada que no está en dicha matriz (y que es 1)
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        #Se calcula el factor beta para obtener la matriz de reflexión
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        #Aquí se va acumulando el producto de las Qj's para llegar a Q_(n-1)*Q(n-2)*...Q_2*Q_1*Q_0=Q
        #al final del ciclo
        Q[j:m,j:m]=(I[j:m,j:m]-beta*np.outer(v,v))@Q[j:m,j:m]
    #La Q_(n-1) es la matriz más chica y la última que calculamos para generar ceros en la última
    #columna de A, y la Q_1 es la matriz más grande y la primera que calculamos para generar ceros
    #en la 1era columna de A
    return Q

matriz_R.__doc__ ="Descripción:\n"\
"        Función que devuelve la matriz R de la factorización QR de una matriz A,\n"\
"        apartir de la matriz Arv\n"\
"\n"\
"Parametros:\n"\
"        A_r_v   np.ndarray que representa matriz (mxn) que incluye elementos de la matriz R y Q,\n"\
"                de la factorización QR\n"\
"\n"\
"Resultado:\n"\
"        R       np.ndarray que representa matriz Q (mxm) de la factorización A=QR\n"\
"\n"\
"El proceso...


SyntaxError: EOL while scanning string literal (<ipython-input-185-99375f58b084>, line 39)

In [ ]:
# def QT_C(A_r_v,C):
#     """
#     Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
#     params: A_r_v   Matriz (mxn) con la info escencial
#             C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

#     return: M       Matriz con numero al azar
#     """
#     m=A_r_v.shape[0]
#     n=A_r_v.shape[1]
#     QT_por_C=np.eye(m)
#     for j in range(n-1,-1,-1):
#         v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
#         beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
#         QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
#     return QT_por_C

In [ ]:
# def Q_j(A_r_v,j):
#     """
#     Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
#     que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
#     params: A_r_v   Matriz (mxn) con la info escencial
#             C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

#     return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
#     """
#     m=A_r_v.shape[0]
#     n=A_r_v.shape[1]
#     Qj=np.eye(m)
#     v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
#     beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
#     Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
#     return Qj

### Funciones para solución de Sistemas de Ecuaciones Lineales

In [ ]:
# def Solucion_SEL_QR_nxn(A,b):
#     """
#     Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
#     params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
#             b   vector (nx1) constantes del sistema

#     return: x   vector que satisface (Ax=b)
#     """
#     A_r_v=factorizacion_QR(A)
#     m=A_r_v.shape[0]
#     Q=np.transpose(Obtencion_Q(A_r_v))
#     R=Obtencion_R(A_r_v)
# #     n=A_r_v.shape[0]
# #     Q=np.eye(m)
# #     R=copy.copy(A)
# #     for j in range(m):
# #         Qj=Q_j(A_r_v,j)
# #         Q=Q@Qj
# #         R=Q_j(A_r_v,j)@R
#     b_prima=np.transpose(Q)@b
#     x = solve_triangular(R, np.transpose(Q)@b)
#     return x

#### Eliminación por bloques

In [ ]:
# def bloques(A, b=False, n1=False, n2=False):
#     """
#     Esta es la función para la creación de bloques usando un arreglo de numpy
    
#     params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
#             b   vector (nx1) constantes del sistema
#             n1  Numero de renglones que tendrá el 1er bloque
#             n2  Numero de renglones que tendrá el 2do bloque
    
#     return: A11 Fraccion de la matriz dividida
#             A12 Fraccion de la matriz dividida
#             A12 Fraccion de la matriz dividida
#             A12 Fraccion de la matriz dividida
#             b1  Fraccion del vector dividido
#             b2  Fraccion del vector dividido
#     """

#     # Primero definimos el n
#     m,n = A.shape

#     # Condiciones de A
#     # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
#     if  not (n1&n2):
#         n1 = n//2
#         n2 = n - n1
#     # Los bloques deben cumplir la condicion de tamaño
#     elif n1+n1 != n:
#         sys.exit('n1 + n2 debe ser igual a n')
#     else:
#         None

#     # Condiciones de b
#     if  b is False:
#         b1 = None
#         b2 = None
#         print('condicion1')
#     elif len(b) == m:
#         b1 = b[:n1]
#         b2 = b[n1:m]
#     else:
#         sys.exit('los renglones de A y b deben ser del mismo tamaño')

#     A11 = A[:n1,:n1]
#     A12 = A[:n1,n1:n]
#     A21 = A[n1:m,:n1]
#     A22 = A[n1:m,n1:n]

#     return A11,A12,A21,A22,b1,b2

In [ ]:
# def eliminacion_bloques(A,b):
#     """
#     Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
#     params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
#             b   vector (nx1) constantes del sistema
    
#     return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
#             x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
#     """
#     if np.linalg.det(A)==0:
#         sys.exit('A debe ser no singular')

#     A11,A12,A21,A22,b1,b2 = bloques(A,b)

#     if np.linalg.det(A11)==0:
#         ys.exit('A11 debe ser no singular')

#     ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
#     ## Aquí se debe usar el método QR una vez que esté desarrollado

#     ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
#     y = Solucion_SEL_QR_nxn(A11,b1)
#     #y = np.linalg.solve(A11,b1)

#     ## Definimos Y = A11^{-1}A12
#     Y = Solucion_SEL_QR_nxn(A11,A12)
#     #Y = np.linalg.solve(A11,A12)

#     ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
#     S = A22 - A21@Y
#     b_h = b2 - A21@y

#     ## 3. Resolver Sx2 = b_hat
#     x2 = Solucion_SEL_QR_nxn(S,b_h)
#     #x2 = np.linalg.solve(S,b_h)

#     ## 4. Resolver A11x1 = b1-A12X2
#     x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
#     #x1 = np.linalg.solve(A11,b1-A12@x2)

#     return np.concatenate((x1,x2), axis=0)

# EJEMPLOS

**Generación de matrices**

In [ ]:
# #Prueba también definiendo una matriz entrada por entrada, solo asegurate que sus entradas sean
# #tipo flotantes/dobles (pues si son de tipo enteras arrastrarán errores de redondeo considerables)
# m=3
# n=3
# A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
# A

In [ ]:
# #Generar una matriz aleatoria de tamaño definido (renglones y columnas) y valores aleatorios comprendidos en un rango, pudiendo ser sus entradas de tipo enteros o dobles
# m=5
# n=3
# A=np.round(crea_matriz(m,n,6,-6,False),2)
# A

**Ejemplos de factorización de una matriz A**

In [ ]:
# A_r_v = factorizacion_QR(A)
# np.round(A_r_v,4)

In [ ]:
# #Obtención de Q con la función QT_C
# Q=np.transpose(QT_C(A_r_v,np.eye(m)))
# np.round(Q,4)

In [ ]:
# #Así obtenemos R
# R=np.transpose(Q)@A
# np.round(R,4)

In [ ]:
# #Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
# Q=np.eye(m)
# R=copy.copy(A)
# for j in range(n):
#     Qj=Q_j(A_r_v,j)
#     print('\nQ',j,':\n',np.round(Qj,4))
#     Q=Q@Qj
#     R=Q_j(A_r_v,j)@R
#     print('\nAplicando las Qjs a A por la izquierda para obtener R, iteracion ',j,':\n',np.round(R,4))

# print('\n\n\nResultados finales:')
# print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
# print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))

In [ ]:
Obtencion_R(A_r_v)

In [ ]:
# #Se prueba un ejemplo más grande con 10^4 entradas
# m=125
# n=80
# A=np.round(crea_matriz(m,n,10,-10,False),2)
# A_r_v = factorizacion_QR(A)
# A_r_v

**Solucion de SEL**

### A) Sistema con soluciones únicas

In [ ]:
# #Generamos un sistema de ecuaciones lineales
# m=6
# n=6
# A=np.round(crea_matriz(m,n,6,-6,False),2)
# b = np.round(crea_matriz(m,1,6,-6,False),2)
# print("A:",A)
# print("b:",b)

##### Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [ ]:
# np.linalg.solve(A,b)

##### Ahora usemos la factorización QR

In [ ]:
# x=Solucion_SEL_QR_nxn(A,b)
# x

In [ ]:
# #Checamos que sea solucion (debe ser igual al vector b)
# A@x

##### Por último usemos eliminación por bloques con QR

In [ ]:
# eliminacion_bloques(A,b)

### B) Sistema sin solución

### C) Sistema con infinitas soluciones